# Schwarz preconditioners for high order finite elements

Basis functions for high order finite element methods (aka p-version fem) are associated with vertices, edges, faces and cells of the triangulation. Thus, there is a natural way to define block-Jacobi (or block-Gauss-Seidel) preconditioners by combining dofs sitting on the same geometric entities (called nodes).

In [14]:
from ngsolve import *
from ngsolve.la import EigenValues_Preconditioner
from ngsolve.webgui import Draw
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))

In [17]:
def DefineProblem(order, printing=True):
    fes = H1(mesh, order=order)
    u,v = fes.TnT()
    a = BilinearForm(grad(u)*grad(v)*dx+10*u*v*dx).Assemble()
    f = LinearForm(x*y*v*dx).Assemble()

    blocks = []
    freedofs = fes.FreeDofs()
    for v in mesh.vertices:
        blocks.append (fes.GetDofNrs(v))
    for ed in mesh.edges:
        blocks.append (fes.GetDofNrs(ed))
    for fa in mesh.faces:
        blocks.append (fes.GetDofNrs(fa))

    if printing:
        print (blocks)
    pre = a.mat.CreateBlockSmoother(blocks, GS=False)

    return a,f,pre

In [18]:
a,f,pre = DefineProblem(2)

[(0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,), (10,), (11,), (12,), (13,), (14,), (15,), (16,), (17,), (18,), (19,), (20,), (21,), (22,), (23,), (24,), (25,), (26,), (27,), (28,), (29,), (30,), (31,), (32,), (33,), (34,), (35,), (36,), (37,), (38,), (39,), (40,), (41,), (42,), (43,), (44,), (45,), (46,), (47,), (48,), (49,), (50,), (51,), (52,), (53,), (54,), (55,), (56,), (57,), (58,), (59,), (60,), (61,), (62,), (63,), (64,), (65,), (66,), (67,), (68,), (69,), (70,), (71,), (72,), (73,), (74,), (75,), (76,), (77,), (78,), (79,), (80,), (81,), (82,), (83,), (84,), (85,), (86,), (87,), (88,), (89,), (90,), (91,), (92,), (93,), (94,), (95,), (96,), (97,), (98,), (99,), (100,), (101,), (102,), (103,), (104,), (105,), (106,), (107,), (108,), (109,), (110,), (111,), (112,), (113,), (114,), (115,), (116,), (117,), (118,), (119,), (120,), (121,), (122,), (123,), (124,), (125,), (126,), (127,), (128,), (129,), (130,), (131,), (132,), (), (), (), (), (), (), (), (), (), (), (), (

In [19]:
gfu = GridFunction(a.space)
from ngsolve.krylovspace import CGSolver
inv = CGSolver(mat=a.mat, pre=pre, printrates='\r', maxiter=400)
gfu.vec.data = inv * f.vec
Draw (gfu);

CG converged in 53 iterations to residual 3.027236450845933e-14


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

**Exercise:** 
* Plot the dependency of the extremal eigenvalues on the polynomial order $p$.
* Compare time to solution. To get comparable results, plot number of dofs per second. 

In [25]:
a,f,pre = DefineProblem(3, False)
lam = list(EigenValues_Preconditioner(a.mat, pre))
print ("lammin, lammax=", lam[0], lam[-1])

lammin, lammax= 0.06934619745427145 2.718724850945184


## Overlapping blocks

One can improve the condition number by letting blocks overlap. Examples:
* One can assign all dofs associated with one element (including the boundary) to a block. The number of blocks is the number of elements.
* One can combine all dofs on all elements sharing a vertex to blocks. The number of blocks is now the number of vertics.
* One can combine all dofs on all edges sharing a vertex to blocks. The dofs on faces and cells are left as extra blocks.

**Exercise:** 
* Implement these definition of blocks.
* Find some more possibilities.
* Which one gives the best condition number ?
* Which one computes the solution in least time ?

Consult NGSolve documentation:
* [Exploring the mesh topology](https://docu.ngsolve.org/latest/i-tutorials/unit-1.8-meshtopology/meshtopology.html)
* [Building blocks for programming preconditioners](https://docu.ngsolve.org/latest/i-tutorials/unit-2.1.2-blockjacobi/blockjacobi.html)

The $p$-robust analysis for triangular and tetrahedral meshes can be found in:
[Additive Schwarz preconditioning for p-version triangular and tetrahedral finite elements](https://academic.oup.com/imajna/article/28/1/1/674492)